### Import Libraries

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('###########################')

### Import Tables to Local (no need to run this part except first time if the raw data remains unchanged)

In [ ]:
# all keywords and corresponding Keyword_Key

In [ ]:
sql1 = """
select *
from amazon_kws;
"""
keyword_all = pd.read_sql(sql1, engine)

In [ ]:
keyword_all.to_csv("keyword_all.csv")

In [ ]:
# all keywords (with Keyword_Key) descend ordered by search volume

In [ ]:
# inner join amazon_kws and amazon_kw_vol
# descend order by the search volume of keywords
sql2 ="""
select amazon_kws."KEYWORD_KEY", amazon_kws."KEYWORD", amazon_kw_vol."KEYWORD_KEY", amazon_kw_vol."DATE", amazon_kw_vol."MARKET_CODE", amazon_kw_vol."DEVICE_CODE", amazon_kw_vol."EST_KEYWORD_SEARCH_VOLUME"
from amazon_kw_vol
inner join amazon_kws
on amazon_kws."KEYWORD_KEY" = amazon_kw_vol."KEYWORD_KEY"
order by "EST_KEYWORD_SEARCH_VOLUME" desc;
"""
keyword_order_by_volume = pd.read_sql(sql2, engine)

In [ ]:
keyword_order_by_volume.to_csv("keyword_order_by_volume.csv")

In [ ]:
# all keywords (with Keyword_Key) descend ordered by SOV

In [ ]:
# inner join amazon_kws and amazon_aso
# select the records with SOV greater than 0.9
# descend order by SOV of keywords
sql3 = """
select amazon_kws."KEYWORD_KEY", amazon_kws."KEYWORD", amazon_aso."KEYWORD_KEY", amazon_aso."DATE", amazon_aso."MARKET_CODE", amazon_aso."DEVICE_CODE", amazon_aso."EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"
from amazon_aso
inner join amazon_kws
on amazon_kws."KEYWORD_KEY" = amazon_aso."KEYWORD_KEY"
order by "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD" desc;
"""
keyword_order_by_sov = pd.read_sql(sql3, engine)

In [ ]:
keyword_order_by_sov.to_csv("keyword_order_by_sov.csv")

### Read Files from Local

In [ ]:
keyword_all = pd.read_csv('keyword_all.csv').iloc[: , 1:]

In [ ]:
keyword_order_by_volume = pd.read_csv('keyword_order_by_volume.csv').iloc[: , 1:]

In [ ]:
keyword_order_by_sov = pd.read_csv('keyword_order_by_sov.csv').iloc[: , 1:]

### Focus on English Keywords

In [ ]:
# define a function to filter out English Keywords (ascii text)
def filter_eng_keywords(dataframe, column):
    filtered_data = dataframe[dataframe[column].map(lambda x: x.isascii())]
    return filtered_data

In [ ]:
keyword_all_eng = filter_eng_keywords(keyword_all, "KEYWORD")
keyword_order_by_volume_eng = filter_eng_keywords(keyword_order_by_volume, "KEYWORD")
keyword_order_by_sov_eng = filter_eng_keywords(keyword_order_by_sov, "KEYWORD")

In [ ]:
# calculate English percentage in all keywords
percentage = int(len(keyword_all_eng))/int(len(keyword_all))
print('There\'re ', int(len(keyword_all)), 'keywords in total.')
print('There\'re ', int(len(keyword_all_eng)), 'keywords in English.')
print('The percentage of English keywords in all keywords is: ', percentage*100, "%.", sep = "")

### Seperate Two Markets (Andriod & iOS)

In [ ]:
# define a function to seperate different markets
def get_type_table(dataframe, market_type):
    return dataframe[dataframe.MARKET_CODE==market_type].copy()

In [ ]:
# Andriod market
keyword_order_by_volume_eng_android = get_type_table(keyword_order_by_volume_eng, "google-play")
keyword_order_by_sov_eng_android = get_type_table(keyword_order_by_sov_eng, "google-play")

In [ ]:
# iOS market
keyword_order_by_volume_eng_ios = get_type_table(keyword_order_by_volume_eng, "apple-store")
keyword_order_by_sov_eng_ios = get_type_table(keyword_order_by_sov_eng, "apple-store")

### Data Distribution

In [ ]:
# data frequency exploration

In [ ]:
# define a function to counter the number in each specific interval
import matplotlib.pyplot as plt
def find_data_distribution(total_groups, dataframe, column_name, range_max, fig_name):
    data = []
    group_name_list = []
    counter_list = []
    batch_size = int(100)/total_groups
    percentage = []

    for i in range(0, total_groups):
        counter_list.append(0)

        nameRange ="[" +str(i *(100/total_groups) )+" ~ "+str((i+1)*(100/total_groups)) + ")"
        if i == total_groups-1:
            nameRange ="[" +str(i *(100/total_groups) )+" ~ "+str((i+1)*(100/total_groups)) + "]"
        group_name_list.append(nameRange)

    for row in dataframe.itertuples():
        data_row = int(getattr(row, column_name))
        data.append(data_row)

        group = int(getattr(row, column_name)*(100/range_max)/batch_size)
        if group == total_groups:
            group -= 1
        counter_list[group] += 1
    for each_section in counter_list:
        percentage.append(each_section/sum(counter_list)*100)

    df = pd.DataFrame({'x': group_name_list, 'y': percentage})
    df.plot(x = 'x', y = 'y', kind = 'bar', figsize = (20, 10))
    plt.savefig(fig_name+'.jpg', dpi=300)
    plt.show()

In [ ]:
find_data_distribution(20, keyword_order_by_volume_eng_android, "EST_KEYWORD_SEARCH_VOLUME", 100, 'volume_frequency_android')
find_data_distribution(20, keyword_order_by_volume_eng_ios, "EST_KEYWORD_SEARCH_VOLUME", 100, 'volume_frequency_ios')

In [ ]:
find_data_distribution(20, keyword_order_by_sov_eng_android, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 1, 'sov_frequency_android')
find_data_distribution(20, keyword_order_by_sov_eng_ios, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 1, 'sov_frequency_ios')

In [ ]:
# describe the data
import numpy as np

In [ ]:
keyword_order_by_volume_eng_android["EST_KEYWORD_SEARCH_VOLUME"].describe()

In [ ]:
high_volume_android_thd = np.percentile(keyword_order_by_volume_eng_android["EST_KEYWORD_SEARCH_VOLUME"], 90)
high_volume_android_thd

In [ ]:
mid_volume_android_thd = np.percentile(keyword_order_by_volume_eng_android["EST_KEYWORD_SEARCH_VOLUME"], 75)
mid_volume_android_thd

In [ ]:
keyword_order_by_volume_eng_ios["EST_KEYWORD_SEARCH_VOLUME"].describe()

In [ ]:
high_volume_ios_thd = np.percentile(keyword_order_by_volume_eng_ios["EST_KEYWORD_SEARCH_VOLUME"], 90)
high_volume_ios_thd

In [ ]:
mid_volume_ios_thd = np.percentile(keyword_order_by_volume_eng_ios["EST_KEYWORD_SEARCH_VOLUME"], 75)
mid_volume_ios_thd

In [ ]:
keyword_order_by_sov_eng_android["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"].describe()

In [ ]:
high_sov_android_thd = np.percentile(keyword_order_by_sov_eng_android["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 95)
high_sov_android_thd

In [ ]:
mid_sov_android_thd = np.percentile(keyword_order_by_sov_eng_android["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 75)
mid_sov_android_thd

In [ ]:
low_sov_android_thd = np.percentile(keyword_order_by_sov_eng_android["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 40)
low_sov_android_thd

In [ ]:
keyword_order_by_sov_eng_ios["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"].describe()

In [ ]:
high_sov_ios_thd = np.percentile(keyword_order_by_sov_eng_ios["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 75)
high_sov_ios_thd

In [ ]:
mid_sov_ios_thd = np.percentile(keyword_order_by_sov_eng_ios["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 55)
mid_sov_ios_thd

In [ ]:
mid_sov_ios_thd = np.percentile(keyword_order_by_sov_eng_ios["EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"], 30)
mid_sov_ios_thd

In [ ]:
# draw PDF and CDF tables

In [ ]:
import scipy.stats as stats

def plot_pdf(data, column_name, fig_name, xlabel, max):
    data = data[column_name]
    mean = np.mean(data)
    std = np.std(data)

    fig_data = stats.norm.pdf(data.sort_values(), mean, std)

    plt.figure(figsize=(15, 5))
    plt.plot(data.sort_values(), fig_data)
    plt.xlim([0, max])
    plt.xlabel(xlabel, size=15)
    plt.ylabel('f(x)', size=15)
    plt.grid(True, alpha=1, linestyle="--")
    plt.locator_params(nbins=20, axis='x')
    plt.savefig(fig_name+'.png', dpi=300)
    plt.show()

def plot_cdf(data, column_name, fig_name, xlabel, max):
    data = data[column_name]
    mean = np.mean(data)
    std = np.std(data)

    fig_data = stats.norm.cdf(data.sort_values(), mean, std)

    plt.figure(figsize=(15, 5))
    plt.plot(data.sort_values(), fig_data)
    plt.xlim([0, max])
    plt.xlabel(xlabel, size=15)
    plt.ylabel('F(x)', size=15)
    plt.grid(True, alpha=1, linestyle="--")
    plt.locator_params(nbins=20, axis='x')
    plt.savefig(fig_name+'.png', dpi=300)
    plt.show()

In [ ]:
search_volume_android_pdf = plot_pdf(keyword_order_by_volume_eng_android, "EST_KEYWORD_SEARCH_VOLUME", 'search_volume_android_pdf', 'Search Volume in Android', 100)

In [ ]:
search_volume_android_cdf = plot_cdf(keyword_order_by_volume_eng_android, "EST_KEYWORD_SEARCH_VOLUME", 'search_volume_android_cdf', 'Search Volume in Android', 100)

In [ ]:
search_volume_ios_pdf = plot_pdf(keyword_order_by_volume_eng_ios, "EST_KEYWORD_SEARCH_VOLUME", 'search_volume_ios_pdf', 'Search Volume in iOS', 100)

In [ ]:
search_volume_ios_cdf = plot_cdf(keyword_order_by_volume_eng_ios, "EST_KEYWORD_SEARCH_VOLUME", 'search_volume_ios_cdf', 'Search Volume in iOS', 100)

In [ ]:
sov_android_pdf = plot_pdf(keyword_order_by_sov_eng_android, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 'sov_ios_pdf', 'SOV in Android', 1)

In [ ]:
sov_android_cdf = plot_cdf(keyword_order_by_sov_eng_android, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 'sov_ios_cdf', 'SOV in Android', 1)

In [ ]:
sov_ios_pdf = plot_pdf(keyword_order_by_sov_eng_ios, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 'sov_ios_pdf', 'SOV in iOS', 1)

In [ ]:
sov_ios_cdf = plot_cdf(keyword_order_by_sov_eng_ios, "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 'sov_ios_cdf', 'SOV in iOS', 1)

### Percentage of A Certain Keyword (Focusing on search volume only)

In [ ]:
# define a function to filter out different categories with SIMILAR keywords, aggregate the total search volume of all the similar keywords in one category in the periods, descending ordered by aggregate search volume

In [ ]:
from progressbar import ProgressBar, Percentage, Bar, Timer, ETA

def merge_related_rows(input):
    input_table = input.copy()
    widgets= ['status: ',Percentage(), ' ', Bar('='),' ', Timer(),' ', ETA()]
    bar = ProgressBar(widgets = widgets, maxval=len(input_table)).start()
    for idx, row_comp in enumerate(input_table.itertuples()):
        comp_keyword = getattr(row_comp, "KEYWORD")
        input_table["KEYWORD"] = input_table["KEYWORD"].apply(lambda x:find_include_info(x, comp_keyword))
        bar.update(idx)
    return input_table

def find_include_info(current_str, comp_str):
    if comp_str in current_str and len(comp_str) > 5:
        return comp_str
    else:
        return current_str

def get_grouped_table(input_table, group_column, sort_column):
    input_table["KEYWORD_AGG"] = input_table["KEYWORD"]
    input_table = merge_related_rows(input_table)
    total_val= input_table[sort_column].sum()
    percentage = ((input_table[sort_column] / total_val)*100).round(2)
    input_table["%"] = percentage
    grouped_table = input_table.groupby([group_column], as_index=False).agg({
     sort_column : 'sum',
     '%' : 'sum',
     'KEYWORD_KEY' : 'first',
     'DATE': 'first',
     'MARKET_CODE' : 'first',
     'DEVICE_CODE' : 'first',
     'KEYWORD_AGG': lambda x: list(set(x))}
    )
    return grouped_table.sort_values(by = [sort_column], ascending=False).iloc[0:999].reset_index().drop(columns="index")

In [ ]:
keyword_order_by_volume_eng_android_grouped = get_grouped_table(keyword_order_by_volume_eng_android,"KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")
keyword_order_by_volume_eng_ios_grouped = get_grouped_table(keyword_order_by_volume_eng_ios, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")

In [ ]:
keyword_all_ordered_android = keyword_order_by_volume_eng_android_grouped.sort_values(by=["%"], inplace=False, ascending=[False]).drop(columns=["DATE","MARKET_CODE","DEVICE_CODE"])
keyword_volume_top_android = keyword_all_ordered_android.reset_index(drop = True)

In [ ]:
keyword_all_ordered_ios = keyword_order_by_volume_eng_ios_grouped.sort_values(by=["%"], inplace=False, ascending=[False]).drop(columns=["DATE","MARKET_CODE","DEVICE_CODE"])
keyword_volume_top_ios = keyword_all_ordered_ios.reset_index(drop = True)

In [ ]:
# print(newDataframe)
def get_rows(dataframe, topN, percentage_column_name):
    dataframe = dataframe.reset_index()
    percentage_min = float(dataframe.iloc[topN, dataframe.columns.get_loc(percentage_column_name)])
    rowNum = 0

    # print(dataframe)
    for index, row in enumerate(dataframe.itertuples()):
        if float(getattr(row, "_4")) < float(percentage_min):
            print(float(getattr(row,"_4")), float(percentage_min))
            rowNum = index
            break

    return  dataframe.iloc[0:rowNum]

In [ ]:
from matplotlib import font_manager as fm
from matplotlib import cm

def plot_pie(dataframe, column1, column2, fig_name):
    categories = dataframe[column1].tolist()
    values = dataframe[column2].tolist()

    test_series = pd.Series(values, index=categories)

    labels = test_series.index
    sizes = test_series.values


    fig, axes = plt.subplots(figsize=(30,30),ncols=2)
    ax1, ax2 = axes.ravel()

    colors = cm.rainbow(np.arange(len(sizes))/len(sizes))
    patches, texts, autotexts = ax1.pie(sizes, labels=labels, autopct='%.2f%%', shadow=False, startangle=90, colors=colors)

    ax1.axis('equal')

    proptease = fm.FontProperties()
    proptease.set_size('large')

    plt.setp(autotexts, fontproperties=proptease)
    plt.setp(texts, fontproperties=proptease)

    ax1.set_title('categories', loc='center')

    ax2.axis('off')
    ax2.legend(patches, labels, loc='center left')

    plt.tight_layout()
    plt.savefig(fig_name+'.jpg', dpi=300)
    plt.show()

In [ ]:
keyword_volume_top_android_final = get_rows(keyword_volume_top_android, 20, "%").drop(columns=["index"])
keyword_volume_top_android_final.to_csv("keyword_volume_top_android_final.csv")

In [ ]:
keyword_volume_top_ios_final = get_rows(keyword_volume_top_ios, 20, "%").drop(columns=["index"])
keyword_volume_top_ios_final.to_csv("keyword_volume_top_ios_final.csv")

In [ ]:
plot_pie(keyword_volume_top_android_final, "KEYWORD", "%", "top_volume_percentage_in_selected_android")

In [ ]:
plot_pie(keyword_volume_top_ios_final, "KEYWORD", "%", "top_volume_percentage_in_selected_ios")

### Filter Out Keywords related to Amazon

In [ ]:
# define a function to filter out all the keywords related to Amazon
def filter_amazon_keywords(dataframe, column):
    filtered_amazon = dataframe[dataframe[column].map(lambda x: not x.find("amazon") == -1)]
    return filtered_amazon

In [ ]:
keyword_amazon_search_volume_android = filter_amazon_keywords(keyword_order_by_volume_eng_android, "KEYWORD")
keyword_amazon_search_volume_ios = filter_amazon_keywords(keyword_order_by_volume_eng_ios, "KEYWORD")
keyword_amazon_sov_android = filter_amazon_keywords(keyword_order_by_sov_eng_android, "KEYWORD")
keyword_amazon_sov_ios = filter_amazon_keywords(keyword_order_by_sov_eng_ios, "KEYWORD")

In [ ]:
# for SEARCH VOLUME, define a function to filter out different categories of COMPLETELY SAME Amazon-related keywords, aggregate the SUM search volume of all the same keywords in one category in the periods, descending ordered by aggregate search volume
def get_sum_table(sum_input_table, sum_group_column, sum_sort_column):
     sum_total_val= sum_input_table[sum_sort_column].sum()
     sum_percentage = ((sum_input_table[sum_sort_column] / sum_total_val)*100).round(2)
     sum_input_table["%"] = sum_percentage

     sum_table = sum_input_table.groupby([sum_group_column], as_index=False).agg({
         sum_sort_column : 'sum',
         'KEYWORD_KEY' : 'first',
         'DATE' : 'first',
         'MARKET_CODE' : 'first',
         'DEVICE_CODE' : 'first'}
     )

     return sum_table.sort_values(by = [sum_sort_column], ascending=False).iloc[0:9999].reset_index().drop(columns="index")

In [ ]:
keyword_amazon_search_volume_android_grouped = get_sum_table(keyword_amazon_search_volume_android,"KEYWORD", "EST_KEYWORD_SEARCH_VOLUME").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])
keyword_amazon_search_volume_ios_grouped = get_sum_table(keyword_amazon_search_volume_ios,"KEYWORD", "EST_KEYWORD_SEARCH_VOLUME").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])

In [ ]:
# for SOV, since it is meaningless to calculate the sum of SOV values, define a function to filter out different categories of COMPLETELY SAME Amazon-related keywords, aggregate the MEAN search volume of all the same keywords in one category in the periods, descending ordered by aggregate search volume
def get_mean_table(mean_input_table, mean_group_column, sum_sort_column):

     mean_table = mean_input_table.groupby([mean_group_column], as_index=False).agg({
         sum_sort_column: 'mean',
         'KEYWORD_KEY' : 'first',
         'DATE': 'first',
         'MARKET_CODE' : 'first',
         'DEVICE_CODE' : 'first'}
     )

     return mean_table.sort_values(by = [sum_sort_column], ascending=False).iloc[0:9999].reset_index().drop(columns="index")

In [ ]:
keyword_amazon_sov_android_grouped = get_mean_table(keyword_amazon_sov_android,"KEYWORD", "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])
keyword_amazon_sov_ios_grouped = get_mean_table(keyword_amazon_sov_ios,"KEYWORD", "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])

In [ ]:
# merged corresponding tables in each segment

In [ ]:
merged_amazon_android = pd.merge(keyword_amazon_search_volume_android_grouped, keyword_amazon_sov_android_grouped, on=["KEYWORD"], how='inner')
merged_amazon_android.to_csv("merged_amazon_android.csv")

In [ ]:
merged_amazon_ios = pd.merge(keyword_amazon_search_volume_ios_grouped, keyword_amazon_sov_ios_grouped, on=["KEYWORD"], how='inner')
merged_amazon_ios.to_csv("merged_amazon_ios.csv")

### Get Dataframes in Setting Range

In [ ]:
def get_volume_data(dataframe, column_name, range1, range2):
    volume_df = dataframe.loc[(dataframe[column_name] <= range2) & (dataframe[column_name] >= range1)]
    return volume_df


def get_sov_data(dataframe, column_name, range1, range2):
    sov_df = dataframe.loc[(dataframe[column_name] <= range2) & (dataframe[column_name] >= range1)]
    return sov_df


def merge_data(dataframe1, dataframe2):
    merged_data = pd.merge(dataframe1, dataframe2, on=["KEYWORD"], how='inner')
    return merged_data

In [ ]:
keyword_order_by_volume_eng_android_grouped = get_sum_table(keyword_order_by_volume_eng_android, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"]).rename(columns={"EST_KEYWORD_SEARCH_VOLUME" : "TOTAL_EST_KEYWORD_SEARCH_VOLUME"})
keyword_order_by_volume_eng_android_grouped

In [ ]:
keyword_order_by_sov_eng_android_grouped = get_mean_table(keyword_order_by_sov_eng_android, "KEYWORD", "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD").rename(columns={"EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD" : "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"}).drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])
keyword_order_by_sov_eng_android_grouped

In [ ]:
keyword_order_by_volume_eng_ios_grouped = get_sum_table(keyword_order_by_volume_eng_ios, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME").drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"]).rename(columns={"EST_KEYWORD_SEARCH_VOLUME" : "TOTAL_EST_KEYWORD_SEARCH_VOLUME"})
keyword_order_by_volume_eng_ios_grouped

In [ ]:
keyword_order_by_sov_eng_ios_grouped = get_mean_table(keyword_order_by_sov_eng_ios, "KEYWORD", "EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD").rename(columns={"EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD" : "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD"}).drop(columns=["KEYWORD_KEY","MARKET_CODE","DEVICE_CODE","DATE"])
keyword_order_by_sov_eng_ios_grouped

### Keywords with High Search Volume & High SOV

In [ ]:
high_volume_android = get_volume_data(keyword_order_by_volume_eng_android_grouped, "TOTAL_EST_KEYWORD_SEARCH_VOLUME", 315, 700)
high_sov_android = get_sov_data(keyword_order_by_sov_eng_android_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.7, 1)
high_volume_high_sov_android = merge_data(high_volume_android, high_sov_android)
high_volume_high_sov_android.to_csv("high_volume_high_sov_android.csv")

In [ ]:
high_volume_ios = get_volume_data(keyword_order_by_volume_eng_ios_grouped, "TOTAL_EST_KEYWORD_SEARCH_VOLUME", 315, 700)
high_sov_ios = get_sov_data(keyword_order_by_sov_eng_ios_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.9, 1)
high_volume_high_sov_ios = merge_data(high_volume_ios, high_sov_ios)
high_volume_high_sov_ios.to_csv("high_volume_high_sov_ios.csv")

### Keywords with High Search Volume & Low SOV

In [ ]:
low_sov_android = get_sov_data(keyword_order_by_sov_eng_android_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.2, 0.5)
high_volume_low_sov_android = merge_data(high_volume_android, low_sov_android)
high_volume_low_sov_android.to_csv("high_volume_low_sov_android.csv")

In [ ]:
low_sov_ios = get_sov_data(keyword_order_by_sov_eng_ios_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.2, 0.5)
high_volume_low_sov_ios = merge_data(high_volume_ios, low_sov_ios)
high_volume_low_sov_ios.to_csv("high_volume_low_sov_ios.csv")

### Keywords with Mid Search Volume & Mid SOV

In [ ]:
mid_volume_android = get_volume_data(keyword_order_by_volume_eng_android_grouped, "TOTAL_EST_KEYWORD_SEARCH_VOLUME", 154, 315)
mid_sov_android = get_sov_data(keyword_order_by_sov_eng_android_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.5, 0.7)
mid_volume_mid_sov_android = merge_data(mid_volume_android, mid_sov_android)
mid_volume_mid_sov_android.to_csv("mid_volume_mid_sov_android.csv")

In [ ]:
mid_volume_ios = get_volume_data(keyword_order_by_volume_eng_ios_grouped, "TOTAL_EST_KEYWORD_SEARCH_VOLUME", 154, 315)
mid_sov_ios = get_sov_data(keyword_order_by_sov_eng_ios_grouped, "AVG_EST_KEYWORD_SOV_APP_SHARE_OF_KEYWORD", 0.5, 0.9)
mid_volume_mid_sov_ios = merge_data(mid_volume_ios, mid_sov_ios)
mid_volume_mid_sov_ios.to_csv("mid_volume_mid_sov_ios.csv")

### Fuzzy Match Function

In [ ]:
def readFiles(filename):
    file1 = open(filename, 'r')
    Lines = file1.readlines()

    categories = []
    info = []
    items = []
    newline = True

    # Strips the newline character
    for line in Lines:
        line = line.strip("\n")
        if line == '_':
            info.append(items)
            items = []
            newline = True
        elif newline:
            newline = False
            categories.append(line)
        else:
            items.append(line)
    return categories, info

def categorize_word(filename, data, threshold, column_name1):
    from thefuzz import fuzz
    result_column1 = pd.DataFrame(data=None, columns=["CATEGORY"])
    result_column2 = pd.DataFrame(data=None, columns=["MATCH"])
    result_column3 = pd.DataFrame(data=None, columns=["SIMILARITY"])


    for index, row in enumerate(data.itertuples()):
        string = getattr(row, column_name1)

        best_category = []
        best_comp_string = []
        best_ratio = 0

        categories, info = readFiles(filename)
        for cate_index, category in enumerate(categories):
            for comp_string in info[cate_index]:
                ratio = fuzz.ratio(string, comp_string)
                if ratio > best_ratio and ratio >= threshold:
                    best_category = [category]
                    best_comp_string = [comp_string]
                    best_ratio = ratio
                elif ratio == best_ratio and ratio >= threshold:
                    best_category.append(category)
                    best_comp_string.append(comp_string)

        if best_ratio < threshold:
            best_category = ["Others"]
            best_comp_string = ["NULL"]
        result_column1.loc[index] = str(best_category[0])
        result_column2.loc[index] = str(best_comp_string[0])
        result_column3.loc[index] = str(best_ratio)

    return result_column1, result_column2, result_column3

def insert_category_column(filename, data, threshold, column_name1, column_name2):
    res1, res2, res3 = categorize_word(filename, data, threshold, column_name1)

    data.insert(len(data.columns), "CATEGORY", res1["CATEGORY"])
    data.insert(len(data.columns), "MATCH", res2["MATCH"])
    data.insert(len(data.columns), "SIMILARITY", res3["SIMILARITY"])
    data = data.sort_values(by = ["CATEGORY", column_name2], ascending =[True, False]).reset_index().drop(columns="index")
    return data

def category_volume_percentage(filename, data, threshold, column_name1, column_name2):
    dataframe = insert_category_column(filename, data, threshold, column_name1, column_name2)

    dataframe_final = dataframe.groupby(by=['CATEGORY'])[column_name2].sum().reset_index()
    dataframe_final['%'] = dataframe_final[column_name2]/dataframe_final[column_name2].sum()
    return dataframe_final

In [ ]:
data1 = pd.read_csv("high_volume_high_sov_android.csv").iloc[:, 1:]
res1 = insert_category_column("categories.txt", data1, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res1.to_csv("cate_high_volume_high_sov_android.csv")
res1

In [ ]:
data1_summary = pd.read_csv("high_volume_high_sov_android.csv").iloc[:, 1:]
res1_summary = category_volume_percentage("categories.txt", data1_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res1_summary

In [ ]:
data2 = pd.read_csv("high_volume_high_sov_ios.csv").iloc[:, 1:]
res2 = insert_category_column("categories.txt", data2, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res2.to_csv("cate_high_volume_high_sov_ios.csv")
res2

In [ ]:
data2_summary = pd.read_csv("high_volume_high_sov_ios.csv").iloc[:, 1:]
res2_summary = category_volume_percentage("categories.txt", data2_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res2_summary

In [ ]:
data3 = pd.read_csv("high_volume_low_sov_android.csv").iloc[:, 1:]
res3 = insert_category_column("categories.txt", data3, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res3.to_csv("cate_high_volume_low_sov_android.csv")
res3

In [ ]:
data3_summary = pd.read_csv("high_volume_low_sov_android.csv").iloc[:, 1:]
res3_summary = category_volume_percentage("categories.txt", data3_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res3_summary

In [ ]:
data4 = pd.read_csv("high_volume_low_sov_ios.csv").iloc[:, 1:]
res4 = insert_category_column("categories.txt", data4, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res4.to_csv("cate_high_volume_low_sov_ios.csv")
res4

In [ ]:
data4_summary = pd.read_csv("high_volume_low_sov_ios.csv").iloc[:, 1:]
res4_summary = category_volume_percentage("categories.txt", data4_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res4_summary

In [ ]:
data5 = pd.read_csv("mid_volume_mid_sov_android.csv").iloc[:, 1:]
res5 = insert_category_column("categories.txt", data5, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res5.to_csv("cate_mid_volume_mid_sov_android.csv")
res5

In [ ]:
data5_summary = pd.read_csv("mid_volume_mid_sov_android.csv").iloc[:, 1:]
res5_summary = category_volume_percentage("categories.txt", data5_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res5_summary

In [ ]:
data6 = pd.read_csv("mid_volume_mid_sov_ios.csv").iloc[:, 1:]
res6 = insert_category_column("categories.txt", data6, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res6.to_csv("cate_mid_volume_mid_sov_ios.csv")
res6

In [ ]:
data6_summary = pd.read_csv("mid_volume_mid_sov_ios.csv").iloc[:, 1:]
res6_summary = category_volume_percentage("categories.txt", data6_summary, 80, "KEYWORD", "TOTAL_EST_KEYWORD_SEARCH_VOLUME")
res6_summary

In [ ]:
amazon_android1 = pd.read_csv("keyword_volume_top_android_final.csv").iloc[:, 1:]
cate_amazon_android1 = insert_category_column("categories.txt", amazon_android1, 80, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")
cate_amazon_android1.to_csv("cate_keyword_volume_top_android_final.csv")
cate_amazon_android1

In [ ]:
amazon_android2 = pd.read_csv("keyword_volume_top_android_final.csv").iloc[:, 1:]
cate_amazon_android2 = category_volume_percentage("categories.txt", amazon_android2, 80, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")
cate_amazon_android2

In [ ]:
amazon_ios1 = pd.read_csv("keyword_volume_top_ios_final.csv").iloc[:, 1:]
cate_amazon_ios1 = insert_category_column("categories.txt", amazon_ios1, 80, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")
cate_amazon_ios1.to_csv("cate_keyword_volume_top_ios_final.csv")
cate_amazon_ios1

In [ ]:
amazon_ios2 = pd.read_csv("keyword_volume_top_ios_final.csv").iloc[:, 1:]
cate_amazon_ios2 = category_volume_percentage("categories.txt", amazon_ios2, 80, "KEYWORD", "EST_KEYWORD_SEARCH_VOLUME")
cate_amazon_ios2